In [0]:
%tensorflow_version 1.x
import tensorflow as tf
from keras.datasets import cifar10
import numpy as np
import keras
from keras.layers import AveragePooling2D, Flatten, Dense
from datetime import datetime
from numpy.random import seed
from sklearn.utils import shuffle

seed(1)
tf.random.set_random_seed(seed=2)


batch_size = 32
epochs = 200
data_augmentation = False
num_classes = 10

subtract_pixel_mean = True

n = 3
depth = n*6+2 # DEPTH = 20


# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Input shape
input_shape = x_train.shape[1:]

# Nomalize data
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

# Substract pixel mean
if subtract_pixel_mean:
  x_train_mean = np.mean(x_train, axis=0)
  x_train -= x_train_mean
  x_test -= x_train_mean

print('x_train_shape: ', x_train.shape)
print('train samples: ', x_train.shape[0])
print('test samples: ' , x_test.shape[0])
print('y_train shape: ', y_train.shape)

# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 100:
        lr *= 0.5e-3
    elif epoch > 70:
        lr *= 1e-3
    elif epoch > 50:
        lr *= 1e-2
    elif epoch > 30:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

def conv2d(x, W, strides, padding):
  return tf.nn.conv2d(x, W, strides= strides, padding=padding)

def weights(kernel_size, in_channels, out_channels, name=None):
  return tf.Variable(tf.keras.initializers.he_normal()([kernel_size, kernel_size, in_channels, out_channels]), name=name)

def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
  

  x=inputs
  in_channels = x.get_shape().as_list()[3]
  beta = tf.Variable(tf.zeros([num_filters]), name="beta")
  if conv_first:
    x = conv2d(x,
                weights(kernel_size,in_channels,out_channels = num_filters),
                strides,
                padding = 'SAME')
    if batch_normalization:
      mean, variance = tf.nn.moments(x, axes=[0,1,2])
      x = tf.nn.batch_normalization(x, mean, variance, offset=None, scale=None, variance_epsilon = 0.000)
    if activation is not None:
      x = tf.nn.relu(x + beta)
  else:
    # batch normalization first
    if batch_normalization:
      mean, variance = tf.nn.moments(x, axes=[0,1,2])
      x = tf.nn.batch_normalization(x, mean, variance, offset=None, scale=None, variance_epsilon = 0.0001)
    if activation is not None:
      x = tf.nn.relu(x + beta)
    # Then convolution
    x = conv2d(x)+ beta

  return x



def resnet_v1(input_tensor, depth, num_classes=10):
  """ResNet Version 1 Model builder [a]

  Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
  Last ReLU is after the shortcut connection.
  At the beginning of each stage, the feature map size is halved (downsampled)
  by a convolutional layer with strides=2, while the number of filters is
  doubled. Within each stage, the layers have the same number filters and the
  same number of filters.
  Features maps sizes:
  stage 0: 32x32, 16
  stage 1: 16x16, 32
  stage 2:  8x8,  64
  The Number of parameters of Rest20 is approx 0.27M


  # Arguments
      input_tensor (tensor): shape of input image tensor
      depth (int): number of core convolutional layers
      num_classes (int): number of classes (CIFAR10 has 10)

  # Returns
      model (Model): tensorflow graph
  """
  if (depth - 2) % 6 != 0:
      raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
  # Start model definition.
  num_filters = 16
  num_res_blocks = int((depth - 2) / 6)

  # inputs: placeholder 

  x = resnet_layer(input_tensor)
  # Instantiate the stack of residual units
  for stack in range(3):
      for res_block in range(num_res_blocks):
          strides = 1
          if stack > 0 and res_block == 0:  # first layer but not first stack
              strides = 2  # downsample
          y = resnet_layer(inputs=x,
                            num_filters=num_filters,
                            strides=strides)
          y = resnet_layer(inputs=y,
                            num_filters=num_filters,
                            activation=None)
          if stack > 0 and res_block == 0:  # first layer but not first stack
              # linear projection residual shortcut connection to match
              # changed dims
              x = resnet_layer(inputs=x,
                                num_filters=num_filters,
                                kernel_size=1,
                                strides=strides,
                                activation=None,
                                batch_normalization=False)
          x = keras.layers.add([x, y])
          x = tf.nn.relu(x)
      num_filters *= 2

  # Add classifier on top.
  # v1 does not use BN after last shortcut connection-ReLU
  x = AveragePooling2D(pool_size=8)(x)
  y = Flatten()(x)
  outputs = Dense(num_classes,
                  kernel_initializer='he_normal')(y)

  return outputs

      

      
  


Using TensorFlow backend.


x_train_shape:  (50000, 32, 32, 3)
train samples:  50000
test samples:  10000
y_train shape:  (50000, 1)


In [0]:
def save_graph(name):
  root_logdir = "log"
  logdir = "{}/run-{}/".format(root_logdir, name)
  file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

name = 'grafo_RESNET20'
save_graph(name)


In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6006 &')

!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-03-10 19:43:15--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.7.1.159, 34.235.245.236, 34.238.36.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.7.1.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.3’

ngrok-stable-linux- 100%[===================>]  13.13M  13.3MB/s    in 1.0s    

2020-03-10 19:43:16 (13.3 MB/s) - ‘ngrok-stable-linux-amd64.zip.3’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
https://f641d10f.ngrok.io


In [0]:
height, width, channels = input_shape

def crear_batch(x_train, y_train, i, batch_size):
    x_batch = x_train[i*batch_size: (i+1)*batch_size]
    y_batch = y_train[i*batch_size:(i+1)*batch_size]
    return x_batch, y_batch

def evaluar_test(x_test,y_test,x,y,lr, lr_value,accuracy,cost, k=10):
  n = len(x_test)
  batch = int(n/k)
  accuracy_test = 0
  loss_test = 0
  for i in range(k):
    x_batch = x_test[i*batch:(i+1)*batch]
    y_batch = y_test[i*batch:(i+1)*batch]
    feed_dict_batch_eval = {x: x_batch, y: y_batch, lr: lr_value} 
    
    accuracy_test += accuracy.eval(feed_dict_batch_eval)/k
    loss_test += cost.eval(feed_dict_batch_eval)/k
  return accuracy_test, loss_test

def get_variable_value(tensor_name, sess):
  t = tf.get_default_graph().get_tensor_by_name(tensor_name)
  r = sess.run(t)
  return r


def train_cnn(epochs,x_train,y_train, checkpoint=None):    
    tf.reset_default_graph()    
    # Entrada grafo
    start = datetime.now()  
    

    x = tf.placeholder(tf.float32, shape=(None, height, width, channels))
    y = tf.placeholder(tf.float32, [None, num_classes])
    lr = tf.placeholder(tf.float32, [])
    output = resnet_v1(x, depth, num_classes=10)
    y_ = tf.nn.softmax(output, name='prediction')
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=y), name='loss')
    optimizer = tf.train.AdamOptimizer(learning_rate = lr).minimize(cost)
    correct = tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))  
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name='accuracy')
    
    # nodo para inicializar variables
    init_op = tf.global_variables_initializer()


    #  Cargar pesos
    if checkpoint == None:
      saver = tf.train.Saver()
    else:
      variables = [v for v in tf.global_variables()]

      vars_to_restore_dict = {}
      for v in variables:
        vars_to_restore_dict[v.name[:-2]] = v
      saver = tf.train.Saver(vars_to_restore_dict)

    #file_writer = tf.summary.FileditareWriter(logdir, tf.get_default_graph())
    vector_costos_train = np.zeros([epochs])
    vector_costos_test = np.zeros([epochs])
    vector_acc_test = np.zeros([epochs])
    vector_acc_train = np.zeros([epochs])
    
    

    with tf.Session() as sess:
        # restaurar pesos
        if checkpoint == None:
          sess.run(init_op) #cuando se realiza inicializacion aleatoria
        else:
          saver.restore(sess, checkpoint) # restaurar desde checkpoint

        # parametros iniciales 
        best_epoch_loss = 10000        
        best_acc_test = 0
        

        
        
        for epoch in range(epochs):
            epoch_loss = 0
            acc_train = 0
            lr_value = lr_schedule(epoch)
            # if random state= None, Shuffle randomly each time.
            x_train, y_train = shuffle(x_train, y_train, random_state=None)        
              

            
            for i in range(int(len(x_train)/batch_size)):
                x_batch, y_batch = crear_batch(x_train,y_train,i,batch_size)
                feed_dict_batch = {x: x_batch, y: y_batch, lr: lr_value}

                _, c = sess.run([optimizer, cost], feed_dict_batch)
                feed_dict_batch_eval = {x: x_batch, y: y_batch, lr: lr_value}              
                acc_batch = accuracy.eval(feed_dict_batch_eval)
                epoch_loss += c
                acc_train += acc_batch
                
            accuracy_test, loss_test =  evaluar_test(x_test,y_test,x,y,lr, lr_value,accuracy,cost,k=10)
            

            print('Epoch %i de %i Loss train: %.4f Loss test: %.4f Accuracy train: %.4f Accuracy test: %.4f '%(epoch,epochs,epoch_loss/(int(len(x_train)/batch_size)),
                                                                                                                   loss_test,
                                                                                                                   acc_train/(int(len(x_train)/batch_size)),
                                                                                                                   accuracy_test))         
            vector_costos_train[epoch] = epoch_loss/(int(len(x_train)/batch_size))
            vector_costos_test[epoch] = loss_test
            vector_acc_train[epoch] = acc_train/(int(len(x_train)/batch_size))        
            vector_acc_test[epoch] = accuracy_test 

            if (accuracy_test > best_acc_test):
              best_acc_test = accuracy_test


              if (epoch%20 == 0):
                save_path = saver.save(sess, "checkpoint{}.ckpt".format(epoch))
            if (epoch%10 == 0):
              difference = datetime.now() - start
              print('Tiempo: ',difference)


        save_path = saver.save(sess,"my_model_final.ckpt")
        difference = datetime.now() - start
        print('Tiempo: ',difference)
        print('El mejor accuracy en test es: ',best_acc_test)
      
        return vector_costos_train, vector_costos_test, vector_acc_test, vector_acc_train

epochs = 100
train_cnn(epochs,x_train,y_train,checkpoint=None)




Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Learning rate:  0.001
Epoch 0 de 100 Loss train: 1.3338 Loss test: 0.9679 Accuracy train: 0.5519 Accuracy test: 0.6547 
Tiempo:  0:01:45.521389
Learning rate:  0.001
Epoch 1 de 100 Loss train: 0.9054 Loss test: 0.7877 Accuracy train: 0.7089 Accuracy test: 0.7273 
Learning rate:  0.001
Epoch 2 de 100 Loss train: 0.7281 Loss test: 0.6735 Accuracy train: 0.7746 Accuracy test: 0.7655 
Learning rate:  0.001
Epoch 3 de 100 Loss train: 0.6199 Loss test: 0.6237 Accuracy train: 0.8135 Accuracy test: 0.7828 
Learning rate:  0.001
Epoch 4 de 100 Loss train: 0.5406 Loss test: 0.6124 Accuracy train: 0.8424 Accuracy test: 0.7904 
Learning rate:  0.001
Epoch 5 de 100 Loss train: 0.4743 Loss test: 0.5894 Accuracy train: 0.8675 Accuracy test: 0.8025 
Learning rate:  0.001
Epoch 6 de 100 Loss train: 0.4223 Loss te

(array([1.33384981e+00, 9.05369921e-01, 7.28116039e-01, 6.19905423e-01,
        5.40611348e-01, 4.74278541e-01, 4.22305107e-01, 3.69843254e-01,
        3.29785784e-01, 2.86415298e-01, 2.54113483e-01, 2.24112560e-01,
        2.01029631e-01, 1.74344460e-01, 1.60769662e-01, 1.45675276e-01,
        1.31030302e-01, 1.21267690e-01, 1.14659804e-01, 1.02592555e-01,
        1.01464268e-01, 9.26420918e-02, 8.84184317e-02, 8.19119577e-02,
        8.46886064e-02, 7.68456328e-02, 7.35747480e-02, 7.62602972e-02,
        6.91873797e-02, 6.46982228e-02, 6.61366651e-02, 2.65270634e-02,
        1.37174857e-02, 9.10273632e-03, 7.91029138e-03, 6.38902992e-03,
        5.45134426e-03, 4.78559349e-03, 4.19724695e-03, 4.14793363e-03,
        3.96109259e-03, 3.16104703e-03, 3.08417972e-03, 2.89388104e-03,
        2.99820751e-03, 2.51811940e-03, 2.45287902e-03, 2.69363319e-03,
        2.44851880e-03, 2.15253313e-03, 2.35443030e-03, 1.87121169e-03,
        1.66333314e-03, 1.61559730e-03, 1.52655016e-03, 1.412619

In [0]:
!zip -r data_graph my_model_final.*

  adding: my_model_final.ckpt.data-00000-of-00001 (deflated 8%)
  adding: my_model_final.ckpt.index (deflated 61%)
  adding: my_model_final.ckpt.meta (deflated 91%)


In [0]:
from google.colab import files
files.download('data_graph.zip')